In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import nltk
from torchtext.data import Field, Iterator
from konlpy.tag import Kkma
kor_tagger = Kkma()

## Word Similarity

In [2]:
pytorch = torch.Tensor([0.6,-0.2,0.7,0.3,0.7,-0.2,0.1,0.1])
tensorflow = torch.Tensor([0.4,-0.1,0.6,-0.2,0.6,-0.2,0.3,0.4])
cat = torch.Tensor([-0.3,0.2,0.1,0.2,-0.2,0.1,-0.3,0.1])

In [3]:
pytorch.dot(tensorflow) # inner product = word similarity

1.1500000953674316

In [4]:
pytorch.dot(cat)

-0.26999998092651367

In [19]:
F.cosine_similarity(pytorch.view(1,-1),tensorflow.view(1,-1))


 0.8417
[torch.FloatTensor of size 1]

In [20]:
F.cosine_similarity(pytorch.view(1,-1),cat.view(1,-1))


-0.3800
[torch.FloatTensor of size 1]

## nn.Embedding() 

In [5]:
embed = nn.Embedding(7,5) # 총 단어 갯수, 임베딩 시킬 차원
embed.weight

Parameter containing:
-0.9484  0.9435  1.6407 -0.8941 -0.4212
 0.0211 -0.0804  1.2075 -0.2758 -0.7180
-0.5715 -0.1710 -1.5368 -1.0341  0.1989
 1.5158 -0.0567  2.5374  0.5363  1.2500
-0.0119  1.5023 -0.6354 -0.1343  0.2100
 0.8259  0.4782 -0.4916  0.6991  0.4878
 0.7340 -0.6501 -1.1134 -0.7186 -0.4187
[torch.FloatTensor of size 7x5]

In [7]:
apple = torch.LongTensor([2]) # 0은 사과의 인덱스
embed(Variable(apple))

Variable containing:
-0.5715 -0.1710 -1.5368 -1.0341  0.1989
[torch.FloatTensor of size 1x5]

In [10]:
sentence = torch.LongTensor([1,3,5]) # 단어의 시퀀스(문장)
embed(Variable(sentence))

Variable containing:
 0.0211 -0.0804  1.2075 -0.2758 -0.7180
 1.5158 -0.0567  2.5374  0.5363  1.2500
 0.8259  0.4782 -0.4916  0.6991  0.4878
[torch.FloatTensor of size 3x5]

### nn.EmbeddingBag 

각 워드 벡터를 합하거나 평균해서 하나의 벡터를 반환 ==> Sentence Embedding

In [11]:
embedding_sum = nn.EmbeddingBag(7, 5, mode='mean') # mean or sum
sentence = Variable(torch.LongTensor([[1,3,5]])) # 토큰 1,3,5로 이루어진 문장
embedding_sum(sentence)

Variable containing:
-1.1688  0.1674 -0.1329 -0.7239  0.0889
[torch.FloatTensor of size 1x5]

## TODO 

- 다음의 코퍼스를 형태소로 tokenize하고 Vocabulary를 구축하라(Word2index)
- 각 단어를 10차원으로 임베딩하기 위한 nn.Embedding을 선언하라
- "토치"와 "텐서플로우"라는 단어를 Embedding matrix에서 읽어서 두 벡터를 내적하라

In [16]:
corpus = ["안녕하세요 저는 파이토치를 공부중입니다.","파이토치는 딥러닝 라이브러리이다", "파이토치와 유사한 것으로 텐서플로우와 케라스 등이 있다",
         "파이토치는 정말 쉽다","텐서플로우는 구글이 만들었다","페북에서는 파이토치를 만들었다", "파이썬과 쉽게 사용할 수 있는 것이 장점이다",
         "그 중 특히 자연어처리할 때 파이토치가 좋다","하지만 아직 베타 버전인게 파이토치의 단점이다","원래는 루아라는 언어로 만들어진 토치라는 프레임워크였다",
         "텐서플로우나 파이토치는 자동미분 기능을 제공한다", "이를 이용하면 딥러닝 모델을 쉽게 만들 수 있다"]

In [17]:
tokenized = [kor_tagger.morphs(c) for c in corpus]

In [19]:
word2index={}
for sent in tokenized:
    for word in sent:
        if word2index.get(word)==None:
            word2index[word]=len(word2index)

In [20]:
len(word2index)

74

In [21]:
matrix = nn.Embedding(len(word2index),10)

In [22]:
matrix.weight.size()

torch.Size([74, 10])

In [23]:
word2index['토치']

6

In [28]:
pytorch_tensor = Variable(torch.LongTensor([word2index['토치']]))
tensorflow_tensor = Variable(torch.LongTensor([word2index['텐서플로우']]))

In [32]:
pytorch_vector = matrix(pytorch_tensor)
tensorflow_vector = matrix(tensorflow_tensor)

In [34]:
print(pytorch_vector.size(),tensorflow_vector.size())

torch.Size([1, 10]) torch.Size([1, 10])


In [35]:
pytorch_vector.dot(tensorflow_vector)

Variable containing:
-0.2746
[torch.FloatTensor of size 1]